To check the connectivity of phonon modes on volumes, fit phonon modes with volumes using polynomial functions up to 5, and fit equations of state. Similar to the QHA module in CRYSTAL17. Required files & input parameters:
* Harmonic phonon calculation output by CRYSTAL17  
* The name list of harmonic phonon outputs, stored with temperature range in 'HA_input.dat'  

By Spica. Vir., ICL, May 13 - 22. spica.h.zhou@gmail.com

In [3]:
import math
import os
import numpy as np

In [13]:
class Mode:
    """
    mode class - store important information for mode with the same rank.
        rank, int, the rank of the mode object
        npoint, int, number of harmonic frequency calculations
        freq, numpy array, npoint * 1 array of frequencies corresponding to the same mode
        vol, numpy array, npoint * 1 array of volumes of harmonic input
        eigvt, numpy array, npoint * 3 array of corresponding eigenvector
        poly_fit, numpy array, npower * 1 array of fitted polynomials
    """

    def __init__(self, rank=0, freq=[], vol=[], eigvt=[]):
        self.rank = rank
        # Unit: THz
        self.freq = np.array(freq, dtype=float) * 2 * math.pi
        # Unit: A^3
        self.vol = np.array(vol, dtype=float)
        # Unit: A
        self.eigvt = np.array(eigvt, dtype=float)

#     def continuity(self, wtout, threshold=0.4):
#         """
#         Check the continuity of phonon modes (NB. Not sure - re-check needed)
#         """
#         wtout.write('%-15s%6i\n' % ('FREQUENCY #', self.rank))
#         wtout.write('%-15s%15s\n' % ('  VOL (A^3)', 'FREQ (cm^-1)'))
#         for i in range(self.npoint):
#             wtout.write('%2s%-15.4f%15.4f\n' % ('', self.vol[i], self.freq[i]))
#             for j in range(i + 1, self.npoint):
#                 dot_eigvt = np.dot(self.eigvt[i], self.eigvt[j])
#                 if dot_eigvt < threshold:
#                     wtout.write('%s%i%s\n'% (
#                         'Warning: Continuity between the above mode and mode ',
#                         j + 1, ' is problematic.'))

#         wtout.write('\n')
        
#         return
        
    def poly_fit(self, wtout, power=[2, 3, 4]):
        """
        Fit phonon frequency as the polynomial function of volume
        """
        npoint = len(self.vol)
        if max(power) > npoint - 1:
            wtout.write('%s\n' % 'Error: Lack referece data for polynomial fit.')   
            return
        
        self.poly_fit = np.array([], dtype=float)
        self.poly_fit_rsqaure = np.array([], dtype=float)
        
        wtout.write('%-6s%9s%12s%4s%s\n' % ('FREQ #', 'POWER', 'R^2', '',
                                            'COEFF LOW TO HIGH'))
        wtout.write('%6i' % self.rank)
        r_squares = np.array([])
        for i in power:
            if i != power[0]:
                wtout.write('%6s' % '')

            func = np.polynomial.polynomial.Polynomial.fit(self.vol, self.freq, i)
            self.poly_fit = np.append(self.poly_fit, func)
            coeff = func.convert().coef
            ss_res = np.sum((self.freq - func(self.vol))**2)
            ss_tot = np.sum((self.freq - np.mean(self.freq))**2)
            r_square = 1 - ss_res / ss_tot

            wtout.write('%9i%12.6f%4s' % (i, r_square, ''))
            for c in coeff:
                wtout.write('%12.4e' % c)
                
            wtout.write('\n')
            self.poly_fit_rsqaure = np.append(self.poly_fit_rsqaure, r_square)
        
        wtout.write('\n')
        
        return self.poly_fit, self.poly_fit_rsqaure
    

In [2]:
class Rearrange_HAdata:
    """
    Rearrange_HAdata class - read the outputs of harmonic frequency calculations and input setting
    files. 
    """
    def __init__(self, setfilename):
        if os.path.isfile(setfilename):
            file = open(setfilename, "r", errors='ignore')
            self.setdata = file.readlines()
            file.close()
        else:
            print('Error: File not exist.')
            return
        
    def readhaout(self):
        """
        Read the input settings file for harmonic calculation outputs.Return to a name
        list of .out files
        """
        label_f = 'FILES'
        label_e = 'END'
        self.filename = []
        countline = 0
        
        while label_f not in self.setdata[countline]:
            countline += 1
        
        countline += 1
        
        while label_e not in self.setdata[countline]:
            self.filename.append(self.setdata[countline].strip())
            countline += 1

        self.data = []
        for file in self.filename:
            hadata = HAdata(hafilename=file)
            hadata.get_volume()
            hadata.get_mode_block()
            self.data.append(hadata)

        return
    
    def readmode(self):
        self.readhaout()
        nmode_array = np.array([i.nmode for i in self.data], dtype=int)
        if len(np.unique(nmode_array)) > 1:
            print('Error: Different numbers of modes detected.')
            return
        else:
            nmode = np.unique(nmode_array)[0]
        
        self.modes = []
        for m in range(nmode):
            volume = np.array([], dtype=float)
            freq = np.array([], dtype=float)
            for ha in self.data:
                volume = np.append(volume, ha.volume)
                single_freq = ha.file[ha.range[0] + m].strip().split()
                single_freq = float(single_freq[4])
                freq = np.append(freq, single_freq)

            rearrange_mode = Mode(rank=m+1, freq=freq, vol=volume)
            self.modes.append(rearrange_mode)
        
        return

In [4]:
class HAdata:
    """
    HAdata class - exact and store key informations of HA output
    """
    def __init__(self, hafilename):
        if os.path.isfile(hafilename):
            file = open(hafilename, "r", errors='ignore')
            self.file = file.readlines()
            file.close()
        else:
            print('Error: File not exist.')
            return

    def get_mode_block(self):
        label_bg = 'MODES         EIGV          FREQUENCIES     IRREP  IR   INTENS    RAMAN'
        label_ed = 'NORMAL MODES NORMALIZED TO CLASSICAL AMPLITUDES (IN BOHR)'
        countline = 0
        
        while label_bg not in self.file[countline]:
            countline += 1

        range_bg = countline + 2
            
        while label_ed not in self.file[countline]:
            countline += 1
            
        range_ed = countline - 2
        nmode = int(self.file[range_ed].strip().split()[1])
        
        self.nmode = nmode
        self.range = np.array([range_bg, range_ed], dtype=int)
        
        return
    
    def get_volume(self):
        label_vol = 'LATTICE PARAMETERS  (ANGSTROMS AND DEGREES) - PRIMITIVE CELL'
        countline = 0
        
        while label_vol not in self.file[countline]:
            countline += 1
            
        self.volume = float(self.file[countline + 2].strip().split()[-1])
        
        return

In [14]:
a = Rearrange_HAdata(setfilename='HA_input.dat')
a.readmode()
print(a.filename)

wtout = open('fitted_freq.dat', "w")
for mode in a.modes:
    mode.poly_fit(wtout=wtout)
    
wtout.close

['f2-r0freq.out', 'f2-m3freq.out', 'f2-m2freq.out', 'f2-m1freq.out', 'f2-p1freq.out', 'f2-p2freq.out', 'f2-p3freq.out', 'f2-p4freq.out', 'f2-p5freq.out', 'f2-p6freq.out']


/tmp/ipykernel_24539/3142680361.py:65: RuntimeWarning: invalid value encountered in double_scalars
  r_square = 1 - ss_res / ss_tot


<function TextIOWrapper.close()>